In [1]:
import glob
import os

import lightning
import optuna
import torch
from lightning import Trainer
from lightning.pytorch.loggers import TensorBoardLogger
from terratorch.tasks.tiled_inference import TiledInferenceParameters

from implementation.training.utilities import (
    TrainingTaskHyperparameterTuningConfig,
    create_model,
)
from roofsense.bag3d import BAG3DTileStore
from roofsense.inference.tile import TiledInferenceEngine
from roofsense.training.datamodule import TrainingDataModule
from roofsense.training.task import TrainingTask

C:\Documents\RoofSense\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:albumentations.check_version:A new version of Albumentations is available: 2.0.5 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


In [2]:
study_name = "optimization"
optim_log_dirpath = os.path.join(r"C:\Documents\RoofSense\logs\3dgeoinfo", study_name)

study = optuna.load_study(
    study_name="optim", storage=f"sqlite:///{optim_log_dirpath}/storage.db"
)

best_params = study.best_params
# Convert parameter format.
for param in ["lab", "tgi"]:
    best_params[f"append_{param}"] = best_params.pop(param)

config = TrainingTaskHyperparameterTuningConfig(
    # Add constant parameters.
    # Encoder
    encoder="tu-resnet18d",
    zero_init_last=True,
    # Loss
    label_smoothing=0.1,
    # Optimizer
    optimizer="AdamW",
    # LR Scheduler
    lr_scheduler="CosineAnnealingLR",
    **best_params,
)

In [3]:
# Quantitative Performance Evaluation
log_dirpath = os.path.join(os.path.dirname(optim_log_dirpath), "test")

torch.set_float32_matmul_precision("high")
torch.backends.cudnn.allow_tf32 = True

lightning.pytorch.seed_everything(0, workers=True)

model = create_model(config)

# Best Step
best_path = os.path.join(
    optim_log_dirpath, f"version_{study.best_trial.number}", "ckpts", "best.ckpt"
)

task: TrainingTask = TrainingTask.load_from_checkpoint(
    checkpoint_path=best_path, model=model
)

datamodule = TrainingDataModule(
    root=r"C:\Documents\RoofSense\roofsense\dataset",
    append_lab=config.append_lab,
    append_tgi=config.append_tgi,
)

trainer = Trainer(
    logger=TensorBoardLogger(
        save_dir=os.path.join(log_dirpath, "quantitative"), name="best"
    ),
    benchmark=True,
)
trainer.test(task, datamodule=datamodule)

# Last Step
for last_path in glob.glob(
    os.path.join(optim_log_dirpath, "validation", "version_*", "ckpts", "last.ckpt")
):
    task: TrainingTask = TrainingTask.load_from_checkpoint(
        checkpoint_path=last_path, model=model
    )

    datamodule = TrainingDataModule(
        root=r"C:\Documents\RoofSense\roofsense\dataset",
        append_lab=config.append_lab,
        append_tgi=config.append_tgi,
    )

    trainer = Trainer(
        logger=TensorBoardLogger(
            save_dir=os.path.join(log_dirpath, "quantitative"), name="last"
        ),
        benchmark=True,
    )
    trainer.test(task, datamodule=datamodule)

Seed set to 0
INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/resnet18d.ra2_in1k)
INFO:timm.models._hub:[timm/resnet18d.ra2_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Converted input conv conv1.0 pretrained weights from 3 to 10 channel(s)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 6/6 [00:07<00:00,  0.79it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       tst/Accuracy0       │            0.0            │
│       tst/Accuracy1       │    0.9483246207237244     │
│       tst/Accuracy2       │    0.8825544118881226     │
│       tst/Accuracy3       │     0.983590841293335     │
│       tst/Accuracy4       │    0.9209630489349365     │
│       tst/Accuracy5       │    0.5459139943122864     │
│       tst/Accuracy6       │    0.9795562028884888     │
│       tst/Accuracy7       │    0.7573972344398499     │
│       tst/Accuracy8       │    0.7809258699417114     │
│       tst/F1Score0        │            0.0            │
│       tst/F1Score1        │    0.9261103868484497     │
│       tst/F1Score2        │     0.920650839805603     │
│       tst/F1Score3        │    0.9594182968139648     │
│       tst/F1Score4        │    0.9283798336982727     │
│       tst/F1Score5        │    0.5603983998298645     │
│       tst/F1Score6        │    0.9316398501396179     │
│       tst/F1Score7        │    0.7900285124778748     │
│       tst/F1Score8        │    0.7190626263618469     │
│     tst/JaccardIndex0     │            0.0            │
│     tst/JaccardIndex1     │    0.8623887300491333     │
│     tst/JaccardIndex2     │    0.8529684543609619     │
│     tst/JaccardIndex3     │     0.922001838684082     │
│     tst/JaccardIndex4     │    0.8663329482078552     │
│     tst/JaccardIndex5     │    0.3892732262611389     │
│     tst/JaccardIndex6     │    0.8720278739929199     │
│     tst/JaccardIndex7     │    0.6529315114021301     │
│     tst/JaccardIndex8     │    0.5613565444946289     │
│         tst/Loss          │     2.000253438949585     │
│     tst/MacroAccuracy     │    0.8499032258987427     │
│     tst/MacroF1Score      │     0.841961145401001     │
│   tst/MacroJaccardIndex   │    0.7474101781845093     │
│    tst/MacroPrecision     │    0.8368954658508301     │
│      tst/MacroRecall      │    0.8499032258987427     │
│     tst/MicroAccuracy     │    0.9112862944602966     │
│     tst/MicroF1Score      │    0.9112862944602966     │
│   tst/MicroJaccardIndex   │    0.8371129631996155     │
│    tst/MicroPrecision     │    0.9112862944602966     │
│      tst/MicroRecall      │    0.9112862944602966     │
│      tst/Precision0       │            0.0            │
│      tst/Precision1       │    0.9049130082130432     │
│      tst/Precision2       │    0.9621845483779907     │
│      tst/Precision3       │      0.9364053606987      │
│      tst/Precision4       │      0.9359170794487      │
│      tst/Precision5       │    0.5756723284721375     │
│      tst/Precision6       │    0.8881925940513611     │
│      tst/Precision7       │    0.8255981206893921     │
│      tst/Precision8       │    0.6662812232971191     │
│        tst/Recall0        │            0.0            │
│        tst/Recall1        │    0.9483246207237244     │
│        tst/Recall2        │    0.8825544118881226     │
│        tst/Recall3        │     0.983590841293335     │
│        tst/Recall4        │    0.9209630489349365     │
│        tst/Recall5        │    0.5459139943122864     │
│        tst/Recall6        │    0.9795562028884888     │
│        tst/Recall7        │    0.7573972344398499     │
│        tst/Recall8        │    0.7809258699417114     │
└───────────────────────────┴───────────────────────────┘

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 6/6 [00:00<00:00,  6.77it/s]


...

Testing DataLoader 0: 100%|██████████| 6/6 [00:00<00:00,  7.02it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       tst/Accuracy0       │            0.0            │
│       tst/Accuracy1       │    0.9257997274398804     │
│       tst/Accuracy2       │    0.8733131885528564     │
│       tst/Accuracy3       │    0.9636247754096985     │
│       tst/Accuracy4       │    0.9384867548942566     │
│       tst/Accuracy5       │    0.5013024806976318     │
│       tst/Accuracy6       │    0.9166441559791565     │
│       tst/Accuracy7       │    0.8011563420295715     │
│       tst/Accuracy8       │    0.9999714493751526     │
│       tst/F1Score0        │            0.0            │
│       tst/F1Score1        │    0.9210631251335144     │
│       tst/F1Score2        │    0.9179703593254089     │
│       tst/F1Score3        │    0.9661107063293457     │
│       tst/F1Score4        │    0.9186787009239197     │
│       tst/F1Score5        │    0.5082312226295471     │
│       tst/F1Score6        │    0.8722036480903625     │
│       tst/F1Score7        │    0.8522566556930542     │
│       tst/F1Score8        │    0.6406452655792236     │
│     tst/JaccardIndex0     │            0.0            │
│     tst/JaccardIndex1     │    0.8536765575408936     │
│     tst/JaccardIndex2     │    0.8483781814575195     │
│     tst/JaccardIndex3     │    0.9344431161880493     │
│     tst/JaccardIndex4     │    0.8495889902114868     │
│     tst/JaccardIndex5     │    0.3406903147697449     │
│     tst/JaccardIndex6     │    0.7733698487281799     │
│     tst/JaccardIndex7     │    0.7425498962402344     │
│     tst/JaccardIndex8     │    0.4712863266468048     │
│         tst/Loss          │     1.989038109779358     │
│     tst/MacroAccuracy     │    0.8650373816490173     │
│     tst/MacroF1Score      │    0.8246449828147888     │
│   tst/MacroJaccardIndex   │    0.7267478704452515     │
│    tst/MacroPrecision     │    0.8101192712783813     │
│      tst/MacroRecall      │    0.8650373816490173     │
│     tst/MicroAccuracy     │     0.903965175151825     │
│     tst/MicroF1Score      │     0.903965175151825     │
│   tst/MicroJaccardIndex   │    0.8251215815544128     │
│    tst/MicroPrecision     │     0.903965175151825     │
│      tst/MicroRecall      │     0.903965175151825     │
│      tst/Precision0       │            0.0            │
│      tst/Precision1       │    0.9163748025894165     │
│      tst/Precision2       │    0.9674407839775085     │
│      tst/Precision3       │    0.9686095118522644     │
│      tst/Precision4       │    0.8996894955635071     │
│      tst/Precision5       │    0.5153541564941406     │
│      tst/Precision6       │    0.8318729996681213     │
│      tst/Precision7       │    0.9103198051452637     │
│      tst/Precision8       │     0.471292644739151     │
│        tst/Recall0        │            0.0            │
│        tst/Recall1        │    0.9257997274398804     │
│        tst/Recall2        │    0.8733131885528564     │
│        tst/Recall3        │    0.9636247754096985     │
│        tst/Recall4        │    0.9384867548942566     │
│        tst/Recall5        │    0.5013024806976318     │
│        tst/Recall6        │    0.9166441559791565     │
│        tst/Recall7        │    0.8011563420295715     │
│        tst/Recall8        │    0.9999714493751526     │
└───────────────────────────┴───────────────────────────┘

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 6/6 [00:00<00:00,  6.34it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       tst/Accuracy0       │            0.0            │
│       tst/Accuracy1       │    0.9338840842247009     │
│       tst/Accuracy2       │    0.8252321481704712     │
│       tst/Accuracy3       │    0.9617123603820801     │
│       tst/Accuracy4       │    0.9409600496292114     │
│       tst/Accuracy5       │    0.5239514112472534     │
│       tst/Accuracy6       │     0.985985279083252     │
│       tst/Accuracy7       │    0.7770686149597168     │
│       tst/Accuracy8       │    0.9993295073509216     │
│       tst/F1Score0        │            0.0            │
│       tst/F1Score1        │    0.9284860491752625     │
│       tst/F1Score2        │    0.8879339098930359     │
│       tst/F1Score3        │    0.9679961204528809     │
│       tst/F1Score4        │    0.9343316555023193     │
│       tst/F1Score5        │    0.5618653893470764     │
│       tst/F1Score6        │    0.8543437719345093     │
│       tst/F1Score7        │    0.8139967322349548     │
│       tst/F1Score8        │    0.6557174324989319     │
│     tst/JaccardIndex0     │            0.0            │
│     tst/JaccardIndex1     │     0.866517961025238     │
│     tst/JaccardIndex2     │     0.798454225063324     │
│     tst/JaccardIndex3     │    0.9379772543907166     │
│     tst/JaccardIndex4     │     0.876756489276886     │
│     tst/JaccardIndex5     │     0.390690416097641     │
│     tst/JaccardIndex6     │    0.7457243800163269     │
│     tst/JaccardIndex7     │    0.6863359808921814     │
│     tst/JaccardIndex8     │    0.48778244853019714    │
│         tst/Loss          │     1.979945182800293     │
│     tst/MacroAccuracy     │    0.8685154318809509     │
│     tst/MacroF1Score      │    0.8255838751792908     │
│   tst/MacroJaccardIndex   │    0.7237799167633057     │
│    tst/MacroPrecision     │    0.8110269904136658     │
│      tst/MacroRecall      │    0.8685154318809509     │
│     tst/MicroAccuracy     │     0.896633505821228     │
│     tst/MicroF1Score      │     0.896633505821228     │
│   tst/MicroJaccardIndex   │    0.8129773139953613     │
│    tst/MicroPrecision     │     0.896633505821228     │
│      tst/MicroRecall      │     0.896633505821228     │
│      tst/Precision0       │            0.0            │
│      tst/Precision1       │    0.9231501221656799     │
│      tst/Precision2       │    0.9609473943710327     │
│      tst/Precision3       │    0.9743625521659851     │
│      tst/Precision4       │    0.9277960062026978     │
│      tst/Precision5       │    0.6056944727897644     │
│      tst/Precision6       │    0.7537135481834412     │
│      tst/Precision7       │    0.8546097278594971     │
│      tst/Precision8       │    0.4879421591758728     │
│        tst/Recall0        │            0.0            │
│        tst/Recall1        │    0.9338840842247009     │
│        tst/Recall2        │    0.8252321481704712     │
│        tst/Recall3        │    0.9617123603820801     │
│        tst/Recall4        │    0.9409600496292114     │
│        tst/Recall5        │    0.5239514112472534     │
│        tst/Recall6        │     0.985985279083252     │
│        tst/Recall7        │    0.7770686149597168     │
│        tst/Recall8        │    0.9993295073509216     │
└───────────────────────────┴───────────────────────────┘

In [4]:
# Qualitative Performance Evaluation
map_dirpath = os.path.join(log_dirpath, "qualitative")
os.makedirs(map_dirpath, exist_ok=True)

tile_id = "9-284-556"

TiledInferenceEngine(
    checkpoint_path=best_path,
    map_location="cpu",
    tile_store=BAG3DTileStore(),
    model=model,
).run(
    tile_id=tile_id,
    dst_filepath=os.path.join(map_dirpath, f"{tile_id}.map.pixel.best.tif"),
    params=TiledInferenceParameters(h_crop=512, h_stride=256, w_crop=512, w_stride=256),
)

Seed set to 0
C:\Documents\RoofSense\roofsense\utilities\file.py:70: UserWarning: The specified path: 'C:\\Documents\\RoofSense\\logs\\3dgeoinfo\\test\\qualitative\\9-284-556.map.pixel.last.tif' does not exist in the system. The value of the 'overwrite' flag will be ignored.
  warnings.warn(msg, UserWarning)
100%|██████████| 200/200 [13:00<00:00,  3.90s/it]
